### Урок 3. Классификация. Логистическая регрессия.

In [250]:
import numpy as np
import matplotlib.pyplot as plt

In [251]:
X = np.array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
              [1, 1, 2, 5, 3, 0, 5, 10, 1, 2],
              [500, 700, 750, 600, 1450, 800, 1500, 2000, 450, 1000],
              [1, 1, 2, 1, 2,  1, 3, 3, 1, 2]], dtype = np.float64)
y = np.array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1], dtype = np.float64)

In [252]:
X

array([[1.00e+00, 1.00e+00, 1.00e+00, 1.00e+00, 1.00e+00, 1.00e+00,
        1.00e+00, 1.00e+00, 1.00e+00, 1.00e+00],
       [1.00e+00, 1.00e+00, 2.00e+00, 5.00e+00, 3.00e+00, 0.00e+00,
        5.00e+00, 1.00e+01, 1.00e+00, 2.00e+00],
       [5.00e+02, 7.00e+02, 7.50e+02, 6.00e+02, 1.45e+03, 8.00e+02,
        1.50e+03, 2.00e+03, 4.50e+02, 1.00e+03],
       [1.00e+00, 1.00e+00, 2.00e+00, 1.00e+00, 2.00e+00, 1.00e+00,
        3.00e+00, 3.00e+00, 1.00e+00, 2.00e+00]])

In [253]:
y

array([0., 0., 1., 0., 1., 0., 1., 0., 1., 1.])

In [254]:
def calc_std_feat(x):
    res = (x - x.mean()) / x.std()
    return res

In [255]:
X_st = X.copy()
X_st[2, :] = calc_std_feat(X[2, :])

In [256]:
X_st

array([[ 1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
         1.        ,  1.        ,  1.        ,  1.        ,  1.        ],
       [ 1.        ,  1.        ,  2.        ,  5.        ,  3.        ,
         0.        ,  5.        , 10.        ,  1.        ,  2.        ],
       [-0.97958969, -0.56713087, -0.46401617, -0.77336028,  0.97958969,
        -0.36090146,  1.08270439,  2.11385144, -1.08270439,  0.05155735],
       [ 1.        ,  1.        ,  2.        ,  1.        ,  2.        ,
         1.        ,  3.        ,  3.        ,  1.        ,  2.        ]])

In [257]:
def calc_logloss(y, y_pred):
    for i in range(len(y_pred)):
        if y_pred[i] == 1:
            y_pred[i] = 0.9999999999
        elif y_pred[i] == 0:
            y_pred[i] = 0.1e-10
    err = - np.mean(y * np.log(y_pred) + (1.0 - y) * np.log(1.0 - y_pred))
    err = np.sum(err)
    return err

In [258]:
# Пример применения
y1 = np.array([1, 0])
y_pred1 = np.array([1., 0.])
# y_pred1 = np.array([1, 0])
calc_logloss(y1, y_pred1)

5.5000004553245406e-11

In [259]:
def sigmoid(z):
    res = 1 / (1 + np.exp(-z))
    return res

Logistic Regression

In [274]:
def eval_model(X, y, iterations, alpha=1e-4):
    np.random.seed(42)
    W = np.random.randn(X.shape[0])
    n = X.shape[1]
    for i in range(1, iterations+1):
        z = np.dot(W, X)
        y_pred = sigmoid(z)
        err = calc_logloss(y, y_pred)
        W -= alpha * (1/n * np.dot((y_pred - y), X.T))
        if i % (iterations / 10) == 0:
            print(i, W, err)
    return W

In [275]:
W = eval_model(X_st, y, iterations=10000, alpha=20)

1000 [-130.13970479  -11.70606768  -37.35163229  107.11970842] 0.17378572882208826
2000 [-139.65055532  -13.28831789  -39.65556779  112.08118075] 0.02466244734563525
3000 [-145.38499234  -10.91653957  -42.51310502  112.49948434] 0.021830807411108905
4000 [-149.19565486  -11.14726189  -43.67440809  115.34324978] 0.02062990350650798
5000 [-152.80800356  -11.37049265  -44.77149386  118.0473891 ] 0.019548840086374714
6000 [-156.2435034   -11.58291385  -45.81476126  120.61938302] 0.018571005189911238
7000 [-159.51795524  -11.78548025  -46.8090288   123.07101597] 0.017682672520262822
8000 [-162.64519367  -11.97902953  -47.75851383  125.41260945] 0.01687239784210577
9000 [-165.63739291  -12.16429993  -48.66692653  127.65324839] 0.01613057134271615
10000 [-168.50531639  -12.34194503  -49.53754637  129.80096672] 0.01544906991747444


In [277]:
def calc_pred_proba(W, X):
    y_pred_proba = 1 / (1 + np.exp(-np.dot(W,X)))
    return y_pred_proba

In [278]:
calc_pred_proba(W, X_st)

array([7.44800051e-02, 1.07662177e-10, 1.00000000e+00, 1.06832659e-27,
       9.96105595e-01, 9.02207450e-10, 1.00000000e+00, 7.19035899e-04,
       9.30099866e-01, 1.00000000e+00])

In [284]:
def calc_pred(W, X):
    y_pred_proba = 1 / (1 + np.exp(-np.dot(W,X)))
    y_pred = []
    for i in y_pred_proba:
        if i > 0.5:
            y_pred.append(1.)
        else:
            y_pred.append(0.)
    return np.array(y_pred)

In [287]:
y_pred = calc_pred(W, X_st)
y_pred

array([0., 0., 1., 0., 1., 0., 1., 0., 1., 1.])

Accuracy

In [290]:
woohoo = 0
count_y = len(y)
for i in range(len(y)):
    if y[i] == y_pred[i]:
        woohoo += 1
accuracy = woohoo/count_y
accuracy # 1.0 = 100%

1.0

Матрица ошибок

In [298]:
error_matrix = np.array([[0, 0], [0, 0]])
for i in range(len(y)):
    if y[i] == y_pred[i] == 1:   # TP
        error_matrix[0][0] += 1
    elif y[i] == y_pred[i] == 0: # TN
        error_matrix[1][1] += 1
    elif y[i] != y_pred[i] and y[i]== 1: # FN
        error_matrix[1][0] += 1
    else:                       #FP
        error_matrix[0][1] += 1 
error_matrix

array([[5, 0],
       [0, 5]])

Точность и полнота

In [303]:
TP = error_matrix[0][0]
FP = error_matrix[0][1]
TN = error_matrix[1][1]
FN = error_matrix[1][0]

In [304]:
precision = TP/(TP + FP)
recall = TP/(TP + FN)
print(f'Точность = {precision}, полнота = {recall}')

Точность = 1.0, полнота = 1.0


F1 score

In [305]:
F = (2 * precision * recall)/(precision + recall)
F

1.0

## Д/з

 1*. Измените функцию calc_logloss так, чтобы нули по возможности не попадали в np.log.

Поставила костыли, но не работает, если значения y_pred - int (пример [0, 1] (только если значения y_pred задавать как float [0., 1.]). Но, мне кажется, вряд ли попадет int, так как это y_pred - сигмоида.
2. Подберите аргументы функции eval_model для логистической регрессии таким образом, чтобы log loss был минимальным.

Чем больше alpha, тем меньше значение log loss. Например iterations=240, alpha=30000. Log loss на последней итерации  = 5.5000004553245406e-11. При этом веса тоже большие. Более менее адекватые, с моей точки зрения, веса и значения ошибки сохранились при параметрах, к примеру, iterations=10000, alpha=20)

3. Создайте функцию calc_pred_proba, возвращающую предсказанную вероятность класса 1 (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred_proba).

Сделано

4. Создайте функцию calc_pred, возвращающую предсказанный класс (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred).

Сделано

5*. Посчитайте Accuracy, матрицу ошибок, точность и полноту, а также F1 score.

Сделано

6. Могла ли модель переобучиться? Почему?

Мне кажется нет, так как здесь представлена линейная разделяющая поверхность